In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 시각화 관련
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter
from PIL import Image
from matplotlib import rc

# 그래프에서 한글 폰트 깨지는 문제에 대한 대처(전역 글꼴 설정)
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action='ignore') 

import matplotlib
matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False

In [7]:
df_origin = pd.read_csv('data/hood_page1to20_final_nodup.csv')
df = df_origin.copy()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35752 entries, 0 to 35751
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   35752 non-null  int64  
 1   user         35752 non-null  object 
 2   gender       33350 non-null  object 
 3   height       33350 non-null  object 
 4   weight       33350 non-null  object 
 5   item         35752 non-null  object 
 6   size         35752 non-null  object 
 7   star         35752 non-null  float64
 8   content      35752 non-null  object 
 9   size_eval    35183 non-null  object 
 10  bright_eval  35183 non-null  object 
 11  color_eval   35183 non-null  object 
 12  thick_eval   35183 non-null  object 
 13  cm           21905 non-null  object 
 14  총장           21905 non-null  float64
 15  어깨너비         20885 non-null  float64
 16  가슴단면         21905 non-null  float64
 17  소매길이         21798 non-null  float64
dtypes: float64(5), int64(1), object(12)
memory usa

In [18]:
print('null값모두삭제:{}'.format(df.dropna(axis = 0).shape))
print('성별,키,몸무게 null 삭제:{}'.format(df[~df['gender'].isnull()&~df['height'].isnull()&~df['weight'].isnull()].shape))
print('성별,키,몸무게 nonnull & 사이즈 하나라도 nonnull:{}'.format(df[~df['gender'].isnull()&~df['height'].isnull()&~df['weight'].isnull()&
                                      (~df['총장'].isnull()|
                                       ~df['어깨너비'].isnull()|~df['가슴단면'].isnull()|
                                       ~df['소매길이'].isnull())].shape))
print('성별,키,몸무게 null:{}'.format(df[df['gender'].isnull()&df['height'].isnull()&df['weight'].isnull()].shape))
print('총장,어깨너비,가슴단면,소매길이4개 전부 null:{}'.format(df[df['총장'].isnull()&
   df['어깨너비'].isnull()&df['가슴단면'].isnull()&
   df['소매길이'].isnull()].shape))
print('총장,어깨너비,가슴단면,소매길이 하나라도 null:{}'.format(df[df['총장'].isnull()|
   df['어깨너비'].isnull()|df['가슴단면'].isnull()|
   df['소매길이'].isnull()].shape))



null값모두삭제:(19511, 18)
성별,키,몸무게 null 삭제:(33350, 18)
성별,키,몸무게 nonnull & 사이즈 하나라도 nonnull:(20467, 18)
성별,키,몸무게 null:(2402, 18)
총장,어깨너비,가슴단면,소매길이4개 전부 null:(13847, 18)
총장,어깨너비,가슴단면,소매길이 하나라도 null:(14967, 18)


In [20]:
df_figure = df[~df['gender'].isnull()&~df['height'].isnull()&~df['weight'].isnull()&
                                      (~df['총장'].isnull()|
                                       ~df['어깨너비'].isnull()|~df['가슴단면'].isnull()|
                                       ~df['소매길이'].isnull())]
df_figure.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20467 entries, 100 to 35737
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   20467 non-null  int64  
 1   user         20467 non-null  object 
 2   gender       20467 non-null  object 
 3   height       20467 non-null  object 
 4   weight       20467 non-null  object 
 5   item         20467 non-null  object 
 6   size         20467 non-null  object 
 7   star         20467 non-null  float64
 8   content      20467 non-null  object 
 9   size_eval    20467 non-null  object 
 10  bright_eval  20467 non-null  object 
 11  color_eval   20467 non-null  object 
 12  thick_eval   20467 non-null  object 
 13  cm           20467 non-null  object 
 14  총장           20467 non-null  float64
 15  어깨너비         19611 non-null  float64
 16  가슴단면         20467 non-null  float64
 17  소매길이         20366 non-null  float64
dtypes: float64(5), int64(1), object(12)
memory u

In [24]:
df_figure.iloc[: , 1:].to_csv('data/hoodtrain_sample.csv',index=False,encoding = "UTF-8")